# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [66]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [67]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,-1.22,72,31,1.72,RU,1717781155
1,1,ghanzi,-21.5667,21.7833,12.10,30,0,3.81,BW,1717781155
2,2,stanley,54.8680,-1.6985,14.87,70,20,7.20,GB,1717781070
3,3,kodiak,57.7900,-152.4072,6.73,81,75,6.17,US,1717781155
4,4,ushuaia,-54.8000,-68.3000,1.79,75,0,2.06,AR,1717781023


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [68]:
##%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    frame_width = 700,
    frame_height = 500,
    scale = 0.9,
    color = "City",
    tiles = True,
    alpha = 0.5
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [69]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_criteria_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_criteria_df.dropna()

# Display sample data
ideal_criteria_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
99,99,baker city,44.7749,-117.8344,24.22,40,0,2.57,US,1717781170
114,114,colonia,50.9333,6.9500,22.20,39,0,3.09,DE,1717780882
122,122,maintirano,-18.0667,44.0167,25.67,75,0,2.70,MG,1717781173
211,211,quelimane,-17.8786,36.8883,24.97,83,0,2.14,MZ,1717781188
217,217,constantia,44.1833,28.6500,24.38,69,0,3.53,RO,1717781189
219,219,ampanihy,-24.7000,44.7500,21.01,61,0,2.79,MG,1717781190
227,227,brookings,44.3114,-96.7984,22.92,39,0,1.54,US,1717781061
300,300,tura,25.5198,90.2201,25.68,77,0,3.33,IN,1717781204
301,301,tsiombe,-25.3000,45.4833,21.51,75,0,1.06,MG,1717781099
316,316,balaklava,44.5000,33.6000,24.65,41,0,2.86,UA,1717780959


### Step 3: Create a new DataFrame called `hotel_df`.

In [70]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_criteria_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
99,99,baker city,44.7749,-117.8344,24.22,40,0,2.57,US,1717781170,
114,114,colonia,50.9333,6.9500,22.20,39,0,3.09,DE,1717780882,
122,122,maintirano,-18.0667,44.0167,25.67,75,0,2.70,MG,1717781173,
211,211,quelimane,-17.8786,36.8883,24.97,83,0,2.14,MZ,1717781188,
217,217,constantia,44.1833,28.6500,24.38,69,0,3.53,RO,1717781189,
219,219,ampanihy,-24.7000,44.7500,21.01,61,0,2.79,MG,1717781190,
227,227,brookings,44.3114,-96.7984,22.92,39,0,1.54,US,1717781061,
300,300,tura,25.5198,90.2201,25.68,77,0,3.33,IN,1717781204,
301,301,tsiombe,-25.3000,45.4833,21.51,75,0,1.06,MG,1717781099,
316,316,balaklava,44.5000,33.6000,24.65,41,0,2.86,UA,1717780959,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [71]:
# Set parameters to search for a hotel
radius = 10,000
params = {
    'apiKey': geoapify_key,
    'api_categories': "accomodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    long = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = "circle:{long},{lat},{radius}"
    params["bias"] = "proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
baker city - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
maintirano - nearest hotel: No hotel found
quelimane - nearest hotel: No hotel found
constantia - nearest hotel: No hotel found
ampanihy - nearest hotel: No hotel found
brookings - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
balaklava - nearest hotel: No hotel found
novofedorovka - nearest hotel: No hotel found
mar del plata - nearest hotel: No hotel found
dondo - nearest hotel: No hotel found
ibia - nearest hotel: No hotel found
gering - nearest hotel: No hotel found
hailey - nearest hotel: No hotel found
urumqi - nearest hotel: No hotel found
orsk - nearest hotel: No hotel found
feodosiya - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
99,99,baker city,44.7749,-117.8344,24.22,40,0,2.57,US,1717781170,No hotel found
114,114,colonia,50.9333,6.9500,22.20,39,0,3.09,DE,1717780882,No hotel found
122,122,maintirano,-18.0667,44.0167,25.67,75,0,2.70,MG,1717781173,No hotel found
211,211,quelimane,-17.8786,36.8883,24.97,83,0,2.14,MZ,1717781188,No hotel found
217,217,constantia,44.1833,28.6500,24.38,69,0,3.53,RO,1717781189,No hotel found
219,219,ampanihy,-24.7000,44.7500,21.01,61,0,2.79,MG,1717781190,No hotel found
227,227,brookings,44.3114,-96.7984,22.92,39,0,1.54,US,1717781061,No hotel found
300,300,tura,25.5198,90.2201,25.68,77,0,3.33,IN,1717781204,No hotel found
301,301,tsiombe,-25.3000,45.4833,21.51,75,0,1.06,MG,1717781099,No hotel found
316,316,balaklava,44.5000,33.6000,24.65,41,0,2.86,UA,1717780959,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [73]:
%%capture --no-display

# Configure the map plot
hotel_map_df = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    frame_width = 700,
    frame_height = 500,
    scale = 1.0,
    color = "City",
    tiles = True,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map_df

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)